In [12]:
import numpy as np
import pandas as pd
import re
import datetime 

In [33]:
type(numbers)

pandas.core.frame.DataFrame

In [28]:
print(numbers.columns)
print(numbers.index)

Index(['MPAA_rating', 'creative_type', 'domestic_box', 'domestic_release',
       'genre', 'international_box', 'language', 'movie_title',
       'production_budget', 'production_country', 'production_method',
       'running_time', 'source', 'worldwide_box'],
      dtype='object')
RangeIndex(start=0, stop=3073, step=1)


# Data Cleansing

## "Numbers" data

In [13]:
numbers = pd.read_csv('boxoffice.csv')

In [69]:
num_clean = numbers.copy()
num_clean = pd.DataFrame(num_clean)

In [70]:
num_clean

,MPAA_rating,creative_type,domestic_box,domestic_release,genre,international_box,language,movie_title,production_budget,production_country,production_method,running_time,source,worldwide_box
0,R,Science Fiction,"$10,030,875","April 7th, 1989 (Wide) by",Action,NaN,English,Cyborg (1989),DNE,United States,Live Action,86 minutes,Original Screenplay,NaN
1,R,Historical Fiction,"$10,005,969","August 10th, 1990 (Wide) by",Thriller/Suspense,NaN,DNE,The Two Jakes (1990),DNE,United States,Live Action,DNE,Based on Fiction Book/Short Story,NaN
2,R,Contemporary Fiction,"$10,438,504","May 3rd, 1991 (Wide)",Comedy,NaN,DNE,A Rage in Harlem (1991),DNE,"United Kingdom,United States",Live Action,DNE,Based on Fiction Book/Short Story,NaN
3,R,Contemporary Fiction,"$10,555,619","September 18th, 1992 (Wide) by",Comedy,NaN,DNE,Husbands and Wives (1992),DNE,United States,Live Action,DNE,Original Screenplay,NaN
4,R,Contemporary Fiction,"$12,108,480","October 15th, 1993 (Wide) by",Thriller/Suspense,NaN,English,Judgment Night (1993),DNE,United States,Live Action,109 minutes,Original Screenplay,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068,PG,Fantasy,"$241,407,328","May 30th, 2014 (Wide) by",Adventure,"$517,129,407",English,Maleficent (2014),"$180,000,000 (worldwide box office is 4.2 time...",United States,Live Action,97 minutes,Spin-Off,"$758,536,735"
3069,PG-13,Science Fiction,"$337,135,885","November 21st, 2014 (Wide) by",Thriller/Suspense,"$429,439,246",English,The Hunger Games: Mockingjay - Part 1 (2014),"$125,000,000 (worldwide box office is 6.1 time...",United States,Live Action,123 minutes,Based on Fiction Book/Short Story,"$766,575,131"
3070,PG-13,Super Hero,"$333,172,112","August 1st, 2014 (Wide) by",Action,"$439,600,000",English,Guardians of the Galaxy (2014),"$170,000,000 (worldwide box office is 4.5 time...",United States,Animation/Live Action,121 minutes,Based on Comic/Graphic Novel,"$772,772,112"
3071,PG-13,Fantasy,"$255,119,788","December 17th, 2014 (Wide) by",Adventure,"$690,457,833",English,The Hobbit: The Battle of the Five Armies (2014),"$250,000,000 (worldwide box office is 3.8 time...","New Zealand,United States",Animation/Live Action,144 minutes,Based on Fiction Book/Short Story,"$945,577,621"


In [61]:
type(num_clean)

pandas.core.frame.DataFrame

#### Domestic Box, International Box, Worldwide Box clean

In [71]:
num_clean['domestic_box'] = num_clean['domestic_box'].str.lstrip('$')
num_clean['domestic_box'] = num_clean['domestic_box'].str.replace(',','')
num_clean['domestic_box'] = num_clean['domestic_box'].fillna(0)
num_clean['domestic_box'] = num_clean['domestic_box'].astype(int)

In [72]:
num_clean['international_box'] = num_clean['international_box'].str.lstrip('$')
num_clean['international_box'] = num_clean['international_box'].str.replace(',','')
num_clean['international_box'] = num_clean['international_box'].fillna(0)
num_clean['international_box'] = num_clean['international_box'].astype(int)

In [73]:
num_clean['worldwide_box'] = num_clean['worldwide_box'].str.lstrip('$')
num_clean['worldwide_box'] = num_clean['worldwide_box'].str.replace(',','')
num_clean['worldwide_box'] = num_clean['worldwide_box'].fillna(0)
num_clean['worldwide_box'] = num_clean['worldwide_box'].astype(int)

In [74]:
num_clean['worldwide_box'] = num_clean['domestic_box'] + num_clean['international_box']

In [75]:
num_clean

,MPAA_rating,creative_type,domestic_box,domestic_release,genre,international_box,language,movie_title,production_budget,production_country,production_method,running_time,source,worldwide_box
0,R,Science Fiction,10030875,"April 7th, 1989 (Wide) by",Action,0,English,Cyborg (1989),DNE,United States,Live Action,86 minutes,Original Screenplay,10030875
1,R,Historical Fiction,10005969,"August 10th, 1990 (Wide) by",Thriller/Suspense,0,DNE,The Two Jakes (1990),DNE,United States,Live Action,DNE,Based on Fiction Book/Short Story,10005969
2,R,Contemporary Fiction,10438504,"May 3rd, 1991 (Wide)",Comedy,0,DNE,A Rage in Harlem (1991),DNE,"United Kingdom,United States",Live Action,DNE,Based on Fiction Book/Short Story,10438504
3,R,Contemporary Fiction,10555619,"September 18th, 1992 (Wide) by",Comedy,0,DNE,Husbands and Wives (1992),DNE,United States,Live Action,DNE,Original Screenplay,10555619
4,R,Contemporary Fiction,12108480,"October 15th, 1993 (Wide) by",Thriller/Suspense,0,English,Judgment Night (1993),DNE,United States,Live Action,109 minutes,Original Screenplay,12108480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068,PG,Fantasy,241407328,"May 30th, 2014 (Wide) by",Adventure,517129407,English,Maleficent (2014),"$180,000,000 (worldwide box office is 4.2 time...",United States,Live Action,97 minutes,Spin-Off,758536735
3069,PG-13,Science Fiction,337135885,"November 21st, 2014 (Wide) by",Thriller/Suspense,429439246,English,The Hunger Games: Mockingjay - Part 1 (2014),"$125,000,000 (worldwide box office is 6.1 time...",United States,Live Action,123 minutes,Based on Fiction Book/Short Story,766575131
3070,PG-13,Super Hero,333172112,"August 1st, 2014 (Wide) by",Action,439600000,English,Guardians of the Galaxy (2014),"$170,000,000 (worldwide box office is 4.5 time...",United States,Animation/Live Action,121 minutes,Based on Comic/Graphic Novel,772772112
3071,PG-13,Fantasy,255119788,"December 17th, 2014 (Wide) by",Adventure,690457833,English,The Hobbit: The Battle of the Five Armies (2014),"$250,000,000 (worldwide box office is 3.8 time...","New Zealand,United States",Animation/Live Action,144 minutes,Based on Fiction Book/Short Story,945577621


#### Production Budget clean

In [139]:
num_clean2 = num_clean.copy()


In [140]:
num_clean2['production_budget'] = num_clean2['production_budget'].str.lstrip('$')
num_clean2['production_budget'] = num_clean2['production_budget'].str.replace(',','')
num_clean2['production_budget'] = num_clean2['production_budget'].str.split(' \(').str[0]
num_clean2['production_budget'] = num_clean2['production_budget'].str.replace('DNE','NaN')


In [141]:
num_clean2

,MPAA_rating,creative_type,domestic_box,domestic_release,genre,international_box,language,movie_title,production_budget,production_country,production_method,running_time,source,worldwide_box
0,R,Science Fiction,10030875,"April 7th, 1989 (Wide) by",Action,0,English,Cyborg (1989),NaN,United States,Live Action,86 minutes,Original Screenplay,10030875
1,R,Historical Fiction,10005969,"August 10th, 1990 (Wide) by",Thriller/Suspense,0,DNE,The Two Jakes (1990),NaN,United States,Live Action,DNE,Based on Fiction Book/Short Story,10005969
2,R,Contemporary Fiction,10438504,"May 3rd, 1991 (Wide)",Comedy,0,DNE,A Rage in Harlem (1991),NaN,"United Kingdom,United States",Live Action,DNE,Based on Fiction Book/Short Story,10438504
3,R,Contemporary Fiction,10555619,"September 18th, 1992 (Wide) by",Comedy,0,DNE,Husbands and Wives (1992),NaN,United States,Live Action,DNE,Original Screenplay,10555619
4,R,Contemporary Fiction,12108480,"October 15th, 1993 (Wide) by",Thriller/Suspense,0,English,Judgment Night (1993),NaN,United States,Live Action,109 minutes,Original Screenplay,12108480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068,PG,Fantasy,241407328,"May 30th, 2014 (Wide) by",Adventure,517129407,English,Maleficent (2014),180000000,United States,Live Action,97 minutes,Spin-Off,758536735
3069,PG-13,Science Fiction,337135885,"November 21st, 2014 (Wide) by",Thriller/Suspense,429439246,English,The Hunger Games: Mockingjay - Part 1 (2014),125000000,United States,Live Action,123 minutes,Based on Fiction Book/Short Story,766575131
3070,PG-13,Super Hero,333172112,"August 1st, 2014 (Wide) by",Action,439600000,English,Guardians of the Galaxy (2014),170000000,United States,Animation/Live Action,121 minutes,Based on Comic/Graphic Novel,772772112
3071,PG-13,Fantasy,255119788,"December 17th, 2014 (Wide) by",Adventure,690457833,English,The Hobbit: The Battle of the Five Armies (2014),250000000,"New Zealand,United States",Animation/Live Action,144 minutes,Based on Fiction Book/Short Story,945577621


#### Release Date clean

In [142]:
#Create copy
num_clean3 = num_clean2.copy()

In [143]:
#Strip unnecessary text from date field
num_clean3['domestic_release'] = num_clean3['domestic_release'].str.split(' \(').str[0]

In [ ]:
# def datestrip(s):
#     return re.sub(r'')

#### Movie Title

In [144]:
#Create copy
num_clean4 = num_clean3.copy()

In [145]:
#Create two columns with original data
movieyear = num_clean4['movie_title'].str.split(' \(',n=1,expand=True)
movieyear[1] = movieyear[1].str.rstrip('\)')

In [146]:
# Create column for movie name and year separately. Remove original column
num_clean4['movie_name'] = movieyear[0]
num_clean4['year'] = movieyear[1]
num_clean4.drop(columns = ['movie_title'], inplace=True)

In [147]:
num_clean4

,MPAA_rating,creative_type,domestic_box,domestic_release,genre,international_box,language,production_budget,production_country,production_method,running_time,source,worldwide_box,movie_name,year
0,R,Science Fiction,10030875,"April 7th, 1989",Action,0,English,NaN,United States,Live Action,86 minutes,Original Screenplay,10030875,Cyborg,1989
1,R,Historical Fiction,10005969,"August 10th, 1990",Thriller/Suspense,0,DNE,NaN,United States,Live Action,DNE,Based on Fiction Book/Short Story,10005969,The Two Jakes,1990
2,R,Contemporary Fiction,10438504,"May 3rd, 1991",Comedy,0,DNE,NaN,"United Kingdom,United States",Live Action,DNE,Based on Fiction Book/Short Story,10438504,A Rage in Harlem,1991
3,R,Contemporary Fiction,10555619,"September 18th, 1992",Comedy,0,DNE,NaN,United States,Live Action,DNE,Original Screenplay,10555619,Husbands and Wives,1992
4,R,Contemporary Fiction,12108480,"October 15th, 1993",Thriller/Suspense,0,English,NaN,United States,Live Action,109 minutes,Original Screenplay,12108480,Judgment Night,1993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068,PG,Fantasy,241407328,"May 30th, 2014",Adventure,517129407,English,180000000,United States,Live Action,97 minutes,Spin-Off,758536735,Maleficent,2014
3069,PG-13,Science Fiction,337135885,"November 21st, 2014",Thriller/Suspense,429439246,English,125000000,United States,Live Action,123 minutes,Based on Fiction Book/Short Story,766575131,The Hunger Games: Mockingjay - Part 1,2014
3070,PG-13,Super Hero,333172112,"August 1st, 2014",Action,439600000,English,170000000,United States,Animation/Live Action,121 minutes,Based on Comic/Graphic Novel,772772112,Guardians of the Galaxy,2014
3071,PG-13,Fantasy,255119788,"December 17th, 2014",Adventure,690457833,English,250000000,"New Zealand,United States",Animation/Live Action,144 minutes,Based on Fiction Book/Short Story,945577621,The Hobbit: The Battle of the Five Armies,2014


#### Running Time

In [148]:
num_clean5 = num_clean4.copy()

In [149]:
num_clean5['running_time'] = num_clean5['running_time'].str.replace('DNE','NaN')
num_clean5['running_time'] = num_clean5['running_time'].str.split(' ').str[0]


In [151]:
num_clean5

,MPAA_rating,creative_type,domestic_box,domestic_release,genre,international_box,language,production_budget,production_country,production_method,running_time,source,worldwide_box,movie_name,year
0,R,Science Fiction,10030875,"April 7th, 1989",Action,0,English,NaN,United States,Live Action,86,Original Screenplay,10030875,Cyborg,1989
1,R,Historical Fiction,10005969,"August 10th, 1990",Thriller/Suspense,0,DNE,NaN,United States,Live Action,NaN,Based on Fiction Book/Short Story,10005969,The Two Jakes,1990
2,R,Contemporary Fiction,10438504,"May 3rd, 1991",Comedy,0,DNE,NaN,"United Kingdom,United States",Live Action,NaN,Based on Fiction Book/Short Story,10438504,A Rage in Harlem,1991
3,R,Contemporary Fiction,10555619,"September 18th, 1992",Comedy,0,DNE,NaN,United States,Live Action,NaN,Original Screenplay,10555619,Husbands and Wives,1992
4,R,Contemporary Fiction,12108480,"October 15th, 1993",Thriller/Suspense,0,English,NaN,United States,Live Action,109,Original Screenplay,12108480,Judgment Night,1993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068,PG,Fantasy,241407328,"May 30th, 2014",Adventure,517129407,English,180000000,United States,Live Action,97,Spin-Off,758536735,Maleficent,2014
3069,PG-13,Science Fiction,337135885,"November 21st, 2014",Thriller/Suspense,429439246,English,125000000,United States,Live Action,123,Based on Fiction Book/Short Story,766575131,The Hunger Games: Mockingjay - Part 1,2014
3070,PG-13,Super Hero,333172112,"August 1st, 2014",Action,439600000,English,170000000,United States,Animation/Live Action,121,Based on Comic/Graphic Novel,772772112,Guardians of the Galaxy,2014
3071,PG-13,Fantasy,255119788,"December 17th, 2014",Adventure,690457833,English,250000000,"New Zealand,United States",Animation/Live Action,144,Based on Fiction Book/Short Story,945577621,The Hobbit: The Battle of the Five Armies,2014


## RottenTomatoes data

In [162]:
rotten = pd.read_csv('rottentomatoes.csv')

In [195]:
rottenclean = rotten.copy()
rottenclean

,certified_status,count_critic_review,count_user_review,critic_score,movie_name,user_score
0,Rotten,38.0,"User Ratings: 60,832",53%,The 'Burbs,71%
1,Certified Fresh,47.0,"User Ratings: 32,587",96%,"Sex, Lies, and Videotape",79%
2,Rotten,33.0,"User Ratings: 58,939",15%,"The Karate Kid, Part III",35%
3,Certified Fresh,47.0,"User Ratings: 376,131",79%,Bill and Ted's Excellent Adventure,75%
4,Rotten,14.0,"User Ratings: 8,400",14%,Three Fugitives,50%
...,...,...,...,...,...,...
2381,Certified Fresh,74.0,"User Ratings: 257,014",91%,James and the Giant Peach,66%
2382,Rotten,66.0,"User Ratings: 49,552",29%,Snow Day,27%
2383,Rotten,107.0,"User Ratings: 436,856",42%,Dr. Dolittle 2,26%
2384,Fresh,159.0,"User Ratings: 300,818",72%,Training Day,89%


#### User review count

In [196]:
# Remove "User Ratings:"
rottenclean['count_user_review'] = rottenclean['count_user_review'].str.split(':').str[1]
#Remove "Not yet available"
rottenclean['count_user_review'] = rottenclean['count_user_review'].str.replace('Not yet available','0')
#Remove commas
rottenclean['count_user_review'] = rottenclean['count_user_review'].str.replace(',','')
#Convert to int
rottenclean['count_user_review'] = rottenclean['count_user_review'].astype(int)

# Convert "Not yet available" which was '0' back to integers
rottenclean['count_user_review'] = rottenclean['count_user_review'].replace(0,'NaN')


In [197]:
rottenclean

,certified_status,count_critic_review,count_user_review,critic_score,movie_name,user_score
0,Rotten,38.0,60832,53%,The 'Burbs,71%
1,Certified Fresh,47.0,32587,96%,"Sex, Lies, and Videotape",79%
2,Rotten,33.0,58939,15%,"The Karate Kid, Part III",35%
3,Certified Fresh,47.0,376131,79%,Bill and Ted's Excellent Adventure,75%
4,Rotten,14.0,8400,14%,Three Fugitives,50%
...,...,...,...,...,...,...
2381,Certified Fresh,74.0,257014,91%,James and the Giant Peach,66%
2382,Rotten,66.0,49552,29%,Snow Day,27%
2383,Rotten,107.0,436856,42%,Dr. Dolittle 2,26%
2384,Fresh,159.0,300818,72%,Training Day,89%


#### Convert all other necessary columns to integers

In [224]:
rottenclean2 = rottenclean.copy()

In [225]:
#Convert critic score to int
rottenclean2['critic_score'] = rottenclean2['critic_score'].str.rstrip('\%')
rottenclean2['critic_score'] = rottenclean2['critic_score'].dropna().astype(int)

#Convert count_critic_review to int
rottenclean2['count_critic_review'] = rottenclean2['count_critic_review'].dropna().astype(int)

#Convert user_score to int
rottenclean2['user_score'] = rottenclean2['user_score'].str.rstrip('\%')
rottenclean2['user_score'] = rottenclean2['user_score'].dropna().astype(int)




# rottenclean2['count_critic_review'] = rottenclean2['count_critic_review'].astype(int)

In [226]:
rottenclean2

,certified_status,count_critic_review,count_user_review,critic_score,movie_name,user_score
0,Rotten,38.0,60832,53.0,The 'Burbs,71.0
1,Certified Fresh,47.0,32587,96.0,"Sex, Lies, and Videotape",79.0
2,Rotten,33.0,58939,15.0,"The Karate Kid, Part III",35.0
3,Certified Fresh,47.0,376131,79.0,Bill and Ted's Excellent Adventure,75.0
4,Rotten,14.0,8400,14.0,Three Fugitives,50.0
...,...,...,...,...,...,...
2381,Certified Fresh,74.0,257014,91.0,James and the Giant Peach,66.0
2382,Rotten,66.0,49552,29.0,Snow Day,27.0
2383,Rotten,107.0,436856,42.0,Dr. Dolittle 2,26.0
2384,Fresh,159.0,300818,72.0,Training Day,89.0


In [231]:
rottenclean2.groupby(['certified_status'])['critic_score','user_score'].describe()

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


critic_score                                                \
                        count       mean        std   min   25%   50%   75%   
certified_status                                                              
Certified Fresh         586.0  85.436860   7.208277  70.0  80.0  85.0  91.0   
DNE                       0.0        NaN        NaN   NaN   NaN   NaN   NaN   
Fresh                   393.0  70.839695   9.379050  60.0  64.0  69.0  73.0   
Rotten                 1130.0  33.071681  15.369589   0.0  21.0  34.0  46.0   

                        user_score                                          \
                    max      count       mean        std   min   25%   50%   
certified_status                                                             
Certified Fresh   100.0      586.0  79.418089  11.335940  40.0  74.0  82.0   
DNE                 NaN      113.0  48.884956  22.605211   0.0  32.0  50.0   
Fresh             100.0      391.0  68.639386  13.871771  20.0  60.0  70.0   
Rotten             59.0     1129.0  51.855624  15.609129  12.0  40.0  51.0   

                               
                   75%    max  
certified_status               
Certified Fresh   88.0   98.0  
DNE               63.0  100.0  
Fresh             79.0   96.0  
Rotten            62.0   96.0

## Combine "Numbers" and "RottenTomatoes" tables

rotten_clean2
num_clean5

In [232]:
rotten_clean2.columns()

NameError: name 'rotten_clean2' is not defined

#### Genre split (follow lecture)

#### Production Country & Language split

### Covert all date-time formats necessary

In [ ]:
# Convert release date to date-time

pd.to_datetime(num_clean3['domestic_release'])